In [1]:
import os
import sys
import pandas as pd
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import traceback

Keep only audio in 'audio' folder

In [5]:
# write code to access the raw/audio folder, move the json files to a new folder 'Json' and delete the jpg files
def move_json_files():
    # get the current working directory
    cwd = os.getcwd()
    # get the path to the raw/audio folder
    path = os.path.join(cwd, 'raw', 'audio')
    # get the list of files in the raw/audio folder
    files = os.listdir(path)
    # create a new folder 'Json' in the raw/audio folder
    new_folder = os.path.join(path, 'Json')
    os.makedirs(new_folder, exist_ok=True)
    # iterate through the files in the raw/audio folder
    for file in files:
        # check if the file is a json file
        if file.endswith('.json'):
            # move the json file to the new folder 'Json'
            os.rename(os.path.join(path, file), os.path.join(new_folder, file))
        # check if the file is a jpg file
        if file.endswith('.jpg'):
            # delete the jpg file
            os.remove(os.path.join(path, file))

 

In [6]:
# run move_json_files function
#move_json_files()

Transoform audio to spectograms

In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import traceback
import gc 

#final
def mp3_to_melspectrogram(mp3_file, audio_output_folder, spec_output_folder, trim_seconds=30, 
                          include_legend=False, nmels=512, segment_duration=5, save_audio_segments=True):
    """
    Converts an MP3 file to mel spectrograms and optionally saves the audio segments.

    Parameters:
        mp3_file (str): Path to the MP3 file.
        audio_output_folder (str): Folder to save the audio segments.
        spec_output_folder (str): Folder to save the spectrogram images.
        trim_seconds (int): Number of seconds to trim from both ends of the audio.
        include_legend (bool): Whether to include legends and axes in the spectrogram plots.
        nmels (int): Number of mel bands to generate in the spectrogram.
        segment_duration (int): Duration of each segment in seconds. If None, processes the entire audio.
        save_audio_segments (bool): Whether to save the corresponding audio segments.
    """
    # Ensure both output folders exist
    os.makedirs(audio_output_folder, exist_ok=True)
    os.makedirs(spec_output_folder, exist_ok=True)

    try:
        print(f"Loading file: {mp3_file}")
        
        # Load the MP3 file using PyDub
        audio = AudioSegment.from_file(mp3_file)
        total_duration_ms = len(audio)  # Duration in milliseconds
        total_duration = total_duration_ms / 1000.0  # Convert to seconds
        print(f"File {mp3_file}, Total Duration: {total_duration:.2f} seconds")

        # Check if the file is long enough to trim
        if total_duration <= 2 * trim_seconds:
            print(f"File {mp3_file} is too short to trim {trim_seconds} seconds from both ends. Skipping.")
            return

        # Trim the first and last trim_seconds
        start_trim = trim_seconds * 1000  # Convert to milliseconds
        end_trim = total_duration_ms - trim_seconds * 1000
        audio_trimmed = audio[start_trim:end_trim]

        print(f"Trimmed audio length (ms): {len(audio_trimmed)}")
        
        # Calculate segment length in samples if segment_duration is specified
        sr = audio_trimmed.frame_rate
        samples_per_segment = int(segment_duration * sr) if segment_duration else len(audio_trimmed.get_array_of_samples())
        
        # Convert to NumPy array
        y = np.array(audio_trimmed.get_array_of_samples()).astype(np.float32)
        
        # Convert stereo to mono
        if audio_trimmed.channels == 2:
            y = y.reshape((-1, 2)).mean(axis=1)

        print(f"Sample rate: {sr}, Audio shape after conversion: {y.shape}")

        # Process each segment if segment_duration is specified
        segment_count = 0
        for start_sample in range(0, len(y) - samples_per_segment + 1, samples_per_segment):
            segment = y[start_sample:start_sample + samples_per_segment]
            segment_count += 1

            # Generate the mel spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=nmels)
            D = librosa.power_to_db(mel_spectrogram, ref=np.max)

            # Plot and save the mel spectrogram
            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D, sr=sr, x_axis='time' if include_legend else None, 
                                     y_axis='mel' if include_legend else None, cmap='inferno')

            if include_legend:
                plt.colorbar(format='%+2.0f dB')  # Add color bar if include_legend is True
                plt.title(f'Mel Spectrogram of {os.path.basename(mp3_file)} - Segment {segment_count}')  # Add title
            else:
                plt.axis('off')  # Turn off axes if include_legend is False
                plt.gca().set_position([0, 0, 1, 1])  # Remove any padding/margins

            # Save the spectrogram image
            output_image_path = os.path.join(
                spec_output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{segment_count}.png"
            )
            plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0 if not include_legend else 0.1)
            plt.close()

            print(f"Spectrogram saved to {output_image_path}")

            # Save the corresponding audio segment if required
            if save_audio_segments:
                audio_segment = audio_trimmed[start_sample // sr * 1000:(start_sample + samples_per_segment) // sr * 1000]
                audio_segment_path = os.path.join(
                    audio_output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{segment_count}.mp3"
                )
                audio_segment.export(audio_segment_path, format="mp3")
                print(f"Audio segment saved to {audio_segment_path}")

        if segment_duration is None:
            # Process entire audio as a single segment if segment_duration is not specified
            mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=nmels)
            D = librosa.power_to_db(mel_spectrogram, ref=np.max)

            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D, sr=sr, x_axis='time' if include_legend else None, 
                                     y_axis='mel' if include_legend else None, cmap='inferno')

            if include_legend:
                plt.colorbar(format='%+2.0f dB')
                plt.title(f'Mel Spectrogram of {os.path.basename(mp3_file)}')

            else:
                plt.axis('off')
                plt.gca().set_position([0, 0, 1, 1])

            output_image_path = os.path.join(
                spec_output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_full.png"
            )
            plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0 if not include_legend else 0.1)
            plt.close()

            print(f"Full audio spectrogram saved to {output_image_path}")

    # Explicitly release memory
        del audio, audio_trimmed, y, segment
        gc.collect()

    except Exception as e:
        print(f"Error processing {mp3_file}: {e}")
        traceback.print_exc()  # Print detailed error traceback


def process_all_mp3_in_folder(folder_path, audio_output_folder, spec_output_folder):
    # Ensure the output folders exist
    os.makedirs(audio_output_folder, exist_ok=True)
    os.makedirs(spec_output_folder, exist_ok=True)

    # Loop through all MP3 files in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".mp3"):
            mp3_file = os.path.join(folder_path, file)

            # Check if the first spectrogram segment exists
            first_segment_path = os.path.join(
                spec_output_folder, f"{os.path.splitext(file)[0]}_segment_1.png"
            )
            if os.path.exists(first_segment_path):
                print(f"Skipping {mp3_file}, spectrogram already exists.")
                continue

            print(f"Processing file: {mp3_file}")
            mp3_to_melspectrogram(mp3_file, audio_output_folder, spec_output_folder)

In [ ]:
process_all_mp3_in_folder("../data/raw/1000dataset", "../data/processed/1000dataset_5/audio", "../data/processed/1000dataset_5/audio_specs")

Skipping ../data/raw/1000dataset\000RDCYioLteXcutOjeweY.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\028gJz6O9HREJ0bKUK8gpQ.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\02MTt3icUotvWO6MMeoLaF.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\02xRKYZ1jY4B6utPa59qSf.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\03f7xZmt2wHCIDJBFPK8G4.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\03oCa1emECqILi28HObI4D.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\03THOy58avyW4Ska4NT8H7.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\042Sl6Mn83JHyLEqdK7uI0.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\04ht5irz2s6Q3vuWXkAOTU.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\04sN26COy28wTXYj3dMoiZ.mp3, spectrogram already exists.
Skipping ../data/raw/1000dataset\04UzZSWfw8hkK832P27QLq.mp3, spectrogram already exists.
Skipping ../data/raw/

c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\librosa\feature\spectral.py:2143: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  mel_basis = filters.mel(sr=sr, n_fft=n_fft, **kwargs)


Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3OZr3vo7SmYpn5XqeQEAOM_segment_1.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3OZr3vo7SmYpn5XqeQEAOM_segment_1.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3OZr3vo7SmYpn5XqeQEAOM_segment_2.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3OZr3vo7SmYpn5XqeQEAOM_segment_2.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3OZr3vo7SmYpn5XqeQEAOM_segment_3.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3OZr3vo7SmYpn5XqeQEAOM_segment_3.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3OZr3vo7SmYpn5XqeQEAOM_segment_4.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3OZr3vo7SmYpn5XqeQEAOM_segment_4.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3OZr3vo7SmYpn5XqeQEAOM_segment_5.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3OZr3vo7SmYpn5XqeQEAOM_segment_5.mp3


Traceback (most recent call last):
  File "C:\Users\spbsp\AppData\Local\Temp\ipykernel_25584\3929784052.py", line 130, in mp3_to_melspectrogram
    gc.collect()
NameError: name 'gc' is not defined


File ../data/raw/1000dataset\3qxOR4Y5Qx3b4eC0b0owe1.mp3, Total Duration: 104.58 seconds
Trimmed audio length (ms): 44583
Sample rate: 44100, Audio shape after conversion: (1966110,)
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3qxOR4Y5Qx3b4eC0b0owe1_segment_1.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3qxOR4Y5Qx3b4eC0b0owe1_segment_1.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3qxOR4Y5Qx3b4eC0b0owe1_segment_2.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3qxOR4Y5Qx3b4eC0b0owe1_segment_2.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3qxOR4Y5Qx3b4eC0b0owe1_segment_3.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3qxOR4Y5Qx3b4eC0b0owe1_segment_3.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\3qxOR4Y5Qx3b4eC0b0owe1_segment_4.png
Audio segment saved to ../data/processed/1000dataset_5/audio\3qxOR4Y5Qx3b4eC0b0owe1_segment_4.mp3
Spectrogram saved 

...

Traceback (most recent call last):
  File "C:\Users\spbsp\AppData\Local\Temp\ipykernel_25584\3929784052.py", line 130, in mp3_to_melspectrogram
    gc.collect()
NameError: name 'gc' is not defined


File ../data/raw/1000dataset\5cjVsWqIkBQC7acTRhL0RO.mp3, Total Duration: 182.69 seconds
Trimmed audio length (ms): 122695
Sample rate: 44100, Audio shape after conversion: (5410849,)
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\5cjVsWqIkBQC7acTRhL0RO_segment_1.png
Audio segment saved to ../data/processed/1000dataset_5/audio\5cjVsWqIkBQC7acTRhL0RO_segment_1.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\5cjVsWqIkBQC7acTRhL0RO_segment_2.png
Audio segment saved to ../data/processed/1000dataset_5/audio\5cjVsWqIkBQC7acTRhL0RO_segment_2.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\5cjVsWqIkBQC7acTRhL0RO_segment_3.png
Audio segment saved to ../data/processed/1000dataset_5/audio\5cjVsWqIkBQC7acTRhL0RO_segment_3.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\5cjVsWqIkBQC7acTRhL0RO_segment_4.png
Audio segment saved to ../data/processed/1000dataset_5/audio\5cjVsWqIkBQC7acTRhL0RO_segment_4.mp3
Spectrogram saved

Traceback (most recent call last):
  File "C:\Users\spbsp\AppData\Local\Temp\ipykernel_25584\3929784052.py", line 130, in mp3_to_melspectrogram
    gc.collect()
NameError: name 'gc' is not defined


File ../data/raw/1000dataset\5cqdwnYqRTB5dq9AQtMtFL.mp3, Total Duration: 166.78 seconds
Trimmed audio length (ms): 106784
Sample rate: 44100, Audio shape after conversion: (4709174,)
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\5cqdwnYqRTB5dq9AQtMtFL_segment_1.png
Audio segment saved to ../data/processed/1000dataset_5/audio\5cqdwnYqRTB5dq9AQtMtFL_segment_1.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\5cqdwnYqRTB5dq9AQtMtFL_segment_2.png
Audio segment saved to ../data/processed/1000dataset_5/audio\5cqdwnYqRTB5dq9AQtMtFL_segment_2.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\5cqdwnYqRTB5dq9AQtMtFL_segment_3.png
Audio segment saved to ../data/processed/1000dataset_5/audio\5cqdwnYqRTB5dq9AQtMtFL_segment_3.mp3
Spectrogram saved to ../data/processed/1000dataset_5/audio_specs\5cqdwnYqRTB5dq9AQtMtFL_segment_4.png
Audio segment saved to ../data/processed/1000dataset_5/audio\5cqdwnYqRTB5dq9AQtMtFL_segment_4.mp3
Spectrogram saved

Traceback (most recent call last):
  File "C:\Users\spbsp\AppData\Local\Temp\ipykernel_25584\3929784052.py", line 130, in mp3_to_melspectrogram
    gc.collect()
NameError: name 'gc' is not defined
Exception in thread Thread-14531 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\subprocess.py", line 1499, in _readerthread
    buffer.append(fh.read())
MemoryError


In [ ]:
def mp3_to_melspectrogram(mp3_file, output_folder, trim_seconds=30, include_legend=False, nmels=512, 
                          segment_duration=5, save_audio_segments=True):
    """
    Converts an MP3 file to mel spectrograms and optionally saves the audio segments.

    Parameters:
        mp3_file (str): Path to the MP3 file.
        output_folder (str): Folder to save the spectrogram images and audio segments.
        trim_seconds (int): Number of seconds to trim from both ends of the audio.
        include_legend (bool): Whether to include legends and axes in the spectrogram plots.
        nmels (int): Number of mel bands to generate in the spectrogram.
        segment_duration (int): Duration of each segment in seconds. If None, processes the entire audio.
        save_audio_segments (bool): Whether to save the corresponding audio segments.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    try:
        print(f"Loading file: {mp3_file}")
        
        # Load the MP3 file using PyDub
        audio = AudioSegment.from_file(mp3_file)
        total_duration_ms = len(audio)  # Duration in milliseconds
        total_duration = total_duration_ms / 1000.0  # Convert to seconds
        print(f"File {mp3_file}, Total Duration: {total_duration:.2f} seconds")

        # Check if the file is long enough to trim
        if total_duration <= 2 * trim_seconds:
            print(f"File {mp3_file} is too short to trim {trim_seconds} seconds from both ends. Skipping.")
            return

        # Trim the first and last trim_seconds
        start_trim = trim_seconds * 1000  # Convert to milliseconds
        end_trim = total_duration_ms - trim_seconds * 1000
        audio_trimmed = audio[start_trim:end_trim]

        print(f"Trimmed audio length (ms): {len(audio_trimmed)}")
        
        # Calculate segment length in samples if segment_duration is specified
        sr = audio_trimmed.frame_rate
        samples_per_segment = int(segment_duration * sr) if segment_duration else len(audio_trimmed.get_array_of_samples())
        
        # Convert to NumPy array
        y = np.array(audio_trimmed.get_array_of_samples()).astype(np.float32)
        
        # Convert stereo to mono
        if audio_trimmed.channels == 2:
            y = y.reshape((-1, 2)).mean(axis=1)

        print(f"Sample rate: {sr}, Audio shape after conversion: {y.shape}")

        # Process each segment if segment_duration is specified
        segment_count = 0
        for start_sample in range(0, len(y) - samples_per_segment + 1, samples_per_segment):
            segment = y[start_sample:start_sample + samples_per_segment]
            segment_count += 1

            # Generate the mel spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=nmels)
            D = librosa.power_to_db(mel_spectrogram, ref=np.max)

            # Plot and save the mel spectrogram
            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D, sr=sr, x_axis='time' if include_legend else None, 
                                     y_axis='mel' if include_legend else None, cmap='inferno')

            if include_legend:
                plt.colorbar(format='%+2.0f dB')  # Add color bar if include_legend is True
                plt.title(f'Mel Spectrogram of {os.path.basename(mp3_file)} - Segment {segment_count}')  # Add title
            else:
                plt.axis('off')  # Turn off axes if include_legend is False
                plt.gca().set_position([0, 0, 1, 1])  # Remove any padding/margins

            # Save the spectrogram image
            output_image_path = os.path.join(
                output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{segment_count}.png"
            )
            plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0 if not include_legend else 0.1)
            plt.close()

            print(f"Spectrogram saved to {output_image_path}")

            # Save the corresponding audio segment if required
            if save_audio_segments:
                audio_segment = audio_trimmed[start_sample // sr * 1000:(start_sample + samples_per_segment) // sr * 1000]
                audio_segment_path = os.path.join(
                    output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{segment_count}.mp3"
                )
                audio_segment.export(audio_segment_path, format="mp3")
                print(f"Audio segment saved to {audio_segment_path}")

        if segment_duration is None:
            # Process entire audio as a single segment if segment_duration is not specified
            mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=nmels)
            D = librosa.power_to_db(mel_spectrogram, ref=np.max)

            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D, sr=sr, x_axis='time' if include_legend else None, 
                                     y_axis='mel' if include_legend else None, cmap='inferno')

            if include_legend:
                plt.colorbar(format='%+2.0f dB')
                plt.title(f'Mel Spectrogram of {os.path.basename(mp3_file)}')

            else:
                plt.axis('off')
                plt.gca().set_position([0, 0, 1, 1])

            output_image_path = os.path.join(
                output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_full.png"
            )
            plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0 if not include_legend else 0.1)
            plt.close()

            print(f"Full audio spectrogram saved to {output_image_path}")

    except Exception as e:
        print(f"Error processing {mp3_file}: {e}")
        traceback.print_exc()  # Print detailed error traceback


def process_all_mp3_in_folder(folder_path, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all MP3 files in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".mp3"):
            mp3_file = os.path.join(folder_path, file)
            output_image_path = os.path.join(output_folder, f"{os.path.splitext(file)[0]}_trimmed.png")
            
            # Check if the spectrogram image already exists
            if os.path.exists(output_image_path):
                print(f"Skipping {mp3_file}, spectrogram already exists.")
                continue

            print(f"Processing file: {mp3_file}")
            mp3_to_melspectrogram(mp3_file, output_folder)

In [ ]:
# Example usage for processing multiple MP3 files
folder_path = "../data/raw/1000dataset"  # Folder containing MP3 files
#output_folder = "../data/raw/1000_mel_spectrograms"  # Folder to save spectrogram images
output_folder = "../data/processed/1000_melspec_5seg_png"  # Folder to save segmented spectrogram images
process_all_mp3_in_folder(folder_path, output_folder)


In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import traceback

def mp3_to_melspectrogram(mp3_file, output_folder, trim_seconds=20, include_legend=False):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    try:
        print(f"Loading file: {mp3_file}")
        
        # Load the MP3 file using PyDub
        audio = AudioSegment.from_file(mp3_file)
        total_duration_ms = len(audio)  # Duration in milliseconds
        total_duration = total_duration_ms / 1000.0  # Convert to seconds
        print(f"File {mp3_file}, Total Duration: {total_duration:.2f} seconds")

        # Check if the file is long enough to trim 20 seconds from both ends
        if total_duration <= 2 * trim_seconds:
            print(f"File {mp3_file} is too short to trim {trim_seconds} seconds from both ends. Skipping.")
            return

        # Trim the first and last 20 seconds
        start_trim = trim_seconds * 1000  # Convert to milliseconds
        end_trim = total_duration_ms - trim_seconds * 1000
        audio_trimmed = audio[start_trim:end_trim]

        print(f"Trimmed audio length (ms): {len(audio_trimmed)}")
        
        # Convert to NumPy array
        y = np.array(audio_trimmed.get_array_of_samples()).astype(np.float32)
        print(f"Initial NumPy array shape: {y.shape}")

        # Adjust for stereo audio
        if audio_trimmed.channels == 2:
            print("Converting stereo to mono")
            y = y.reshape((-1, 2))
            y = y.mean(axis=1)  # Convert to mono by averaging channels

        sr = audio_trimmed.frame_rate
        print(f"Sample rate: {sr}, Final NumPy array shape: {y.shape}")

        # Generate the mel spectrogram
        mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
        D = librosa.power_to_db(mel_spectrogram, ref=np.max)

        # Plot the mel spectrogram
        plt.figure(figsize=(10, 6))
        librosa.display.specshow(D, sr=sr, x_axis='time' if include_legend else None, 
                                 y_axis='mel' if include_legend else None, cmap='inferno')

        if include_legend:
            plt.colorbar(format='%+2.0f dB')  # Add color bar if include_legend is True
            plt.title(f'Mel Spectrogram of {os.path.basename(mp3_file)} (trimmed)')  # Add title
        else:
            plt.axis('off')  # Turn off axes if include_legend is False
            plt.gca().set_position([0, 0, 1, 1])  # Remove any padding/margins

        # Save the spectrogram image
        output_image_path = os.path.join(
            output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_trimmed.png"
        )
        plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0 if not include_legend else 0.1)
        plt.close()

        print(f"Spectrogram saved to {output_image_path}")
        return output_image_path

    except Exception as e:
        print(f"Error processing {mp3_file}: {e}")
        traceback.print_exc()  # Print detailed error traceback


In [ ]:
# Example usage for processing multiple MP3 files
folder_path = "../data/raw/1000dataset"  # Folder containing MP3 files
#output_folder = "../data/raw/1000_mel_spectrograms"  # Folder to save spectrogram images
output_folder = "../data/raw/1000_mel_spec_seg"  # Folder to save segmented spectrogram images
process_all_mp3_in_folder(folder_path, output_folder)


legend and segment parameters

In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment
import traceback

def mp3_to_melspectrogram(mp3_file, output_folder, trim_seconds=20, include_legend=False, nmels=128, segment_duration=15):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    try:
        print(f"Loading file: {mp3_file}")
        
        # Load the MP3 file using PyDub
        audio = AudioSegment.from_file(mp3_file)
        total_duration_ms = len(audio)  # Duration in milliseconds
        total_duration = total_duration_ms / 1000.0  # Convert to seconds
        print(f"File {mp3_file}, Total Duration: {total_duration:.2f} seconds")

        # Check if the file is long enough to trim 20 seconds from both ends
        if total_duration <= 2 * trim_seconds:
            print(f"File {mp3_file} is too short to trim {trim_seconds} seconds from both ends. Skipping.")
            return

        # Trim the first and last 20 seconds
        start_trim = trim_seconds * 1000  # Convert to milliseconds
        end_trim = total_duration_ms - trim_seconds * 1000
        audio_trimmed = audio[start_trim:end_trim]

        print(f"Trimmed audio length (ms): {len(audio_trimmed)}")
        
        # Calculate segment length in samples if segment_duration is specified
        sr = audio_trimmed.frame_rate
        samples_per_segment = int(segment_duration * sr) if segment_duration else len(audio_trimmed.get_array_of_samples())
        
        # Convert to NumPy array
        y = np.array(audio_trimmed.get_array_of_samples()).astype(np.float32)
        
        # Convert stereo to mono
        if audio_trimmed.channels == 2:
            y = y.reshape((-1, 2)).mean(axis=1)

        print(f"Sample rate: {sr}, Audio shape after conversion: {y.shape}")

        # Process each segment if segment_duration is specified
        segment_count = 0
        for start_sample in range(0, len(y) - samples_per_segment + 1, samples_per_segment):
            segment = y[start_sample:start_sample + samples_per_segment]
            segment_count += 1

            # Generate the mel spectrogram
            mel_spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=nmels)
            D = librosa.power_to_db(mel_spectrogram, ref=np.max)

            # Plot and save the mel spectrogram
            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D, sr=sr, x_axis='time' if include_legend else None, 
                                     y_axis='mel' if include_legend else None, cmap='inferno')

            if include_legend:
                plt.colorbar(format='%+2.0f dB')  # Add color bar if include_legend is True
                plt.title(f'Mel Spectrogram of {os.path.basename(mp3_file)} - Segment {segment_count}')  # Add title
            else:
                plt.axis('off')  # Turn off axes if include_legend is False
                plt.gca().set_position([0, 0, 1, 1])  # Remove any padding/margins

            # Save the spectrogram image
            output_image_path = os.path.join(
                output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{segment_count}.png"
            )
            plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0 if not include_legend else 0.1)
            plt.close()

            print(f"Spectrogram saved to {output_image_path}")

        if segment_duration is None:
            # Process entire audio as a single segment if segment_duration is not specified
            mel_spectrogram = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=nmels)
            D = librosa.power_to_db(mel_spectrogram, ref=np.max)

            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D, sr=sr, x_axis='time' if include_legend else None, 
                                     y_axis='mel' if include_legend else None, cmap='inferno')

            if include_legend:
                plt.colorbar(format='%+2.0f dB')
                plt.title(f'Mel Spectrogram of {os.path.basename(mp3_file)}')

            else:
                plt.axis('off')
                plt.gca().set_position([0, 0, 1, 1])

            output_image_path = os.path.join(
                output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_full.png"
            )
            plt.savefig(output_image_path, bbox_inches='tight', pad_inches=0 if not include_legend else 0.1)
            plt.close()

            print(f"Full audio spectrogram saved to {output_image_path}")

    except Exception as e:
        print(f"Error processing {mp3_file}: {e}")
        traceback.print_exc()  # Print detailed error traceback
        
        
def process_all_mp3_in_folder(folder_path, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all MP3 files in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".mp3"):
            mp3_file = os.path.join(folder_path, file)
            output_image_path = os.path.join(output_folder, f"{os.path.splitext(file)[0]}_trimmed.png")
            
            # Check if the spectrogram image already exists
            if os.path.exists(output_image_path):
                print(f"Skipping {mp3_file}, spectrogram already exists.")
                continue

            print(f"Processing file: {mp3_file}")
            mp3_to_melspectrogram(mp3_file, output_folder)


In [ ]:
# Example usage for processing multiple MP3 files
folder_path = "../data/raw/1000dataset"  # Folder containing MP3 files
#output_folder = "../data/raw/1000_mel_spectrograms"  # Folder to save spectrogram images
output_folder = "../data/raw/1000_mel_spec_seg"  # Folder to save segmented spectrogram images
process_all_mp3_in_folder(folder_path, output_folder)


Processing file: ../data/raw/1000dataset\000RDCYioLteXcutOjeweY.mp3
Loading file: ../data/raw/1000dataset\000RDCYioLteXcutOjeweY.mp3
File ../data/raw/1000dataset\000RDCYioLteXcutOjeweY.mp3, Total Duration: 214.37 seconds
Trimmed audio length (ms): 174367
Sample rate: 44100, Audio shape after conversion: (7689584,)
Spectrogram saved to ../data/raw/1000_mel_spec_seg\000RDCYioLteXcutOjeweY_segment_1.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\000RDCYioLteXcutOjeweY_segment_2.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\000RDCYioLteXcutOjeweY_segment_3.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\000RDCYioLteXcutOjeweY_segment_4.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\000RDCYioLteXcutOjeweY_segment_5.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\000RDCYioLteXcutOjeweY_segment_6.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\000RDCYioLteXcutOjeweY_segment_7.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\000RDCYioLteXcutO

Traceback (most recent call last):
  File "C:\Users\spbsp\AppData\Local\Temp\ipykernel_48260\3871053562.py", line 54, in mp3_to_melspectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=128)
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\librosa\feature\spectral.py", line 2130, in melspectrogram
    S, n_fft = _spectrogram(
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\librosa\core\spectrum.py", line 2945, in _spectrogram
    stft(
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\librosa\core\spectrum.py", line 355, in stft
    stft_matrix = np.zeros(shape, dtype=dtype, order="F")
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 10.1 MiB for an array with shape (1025, 1292) and data type complex64


File ../data/raw/1000dataset\6320dfS3Vt9DXC5RNwC4oL.mp3, Total Duration: 204.61 seconds
Trimmed audio length (ms): 164614
Sample rate: 44100, Audio shape after conversion: (7259477,)
Spectrogram saved to ../data/raw/1000_mel_spec_seg\6320dfS3Vt9DXC5RNwC4oL_segment_1.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\6320dfS3Vt9DXC5RNwC4oL_segment_2.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\6320dfS3Vt9DXC5RNwC4oL_segment_3.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\6320dfS3Vt9DXC5RNwC4oL_segment_4.png
Spectrogram saved to ../data/raw/1000_mel_spec_seg\6320dfS3Vt9DXC5RNwC4oL_segment_5.png
Error processing ../data/raw/1000dataset\6320dfS3Vt9DXC5RNwC4oL.mp3: Unable to allocate 10.1 MiB for an array with shape (1025, 1292) and data type complex64
Processing file: ../data/raw/1000dataset\63vWEUXhpgghscp9FgvCMG.mp3
Loading file: ../data/raw/1000dataset\63vWEUXhpgghscp9FgvCMG.mp3


Traceback (most recent call last):
  File "C:\Users\spbsp\AppData\Local\Temp\ipykernel_48260\3871053562.py", line 54, in mp3_to_melspectrogram
    mel_spectrogram = librosa.feature.melspectrogram(y=segment, sr=sr, n_mels=128)
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\librosa\feature\spectral.py", line 2130, in melspectrogram
    S, n_fft = _spectrogram(
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\librosa\core\spectrum.py", line 2945, in _spectrogram
    stft(
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\site-packages\librosa\core\spectrum.py", line 355, in stft
    stft_matrix = np.zeros(shape, dtype=dtype, order="F")
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 10.1 MiB for an array with shape (1025, 1292) and data type complex64
Exception in thread Thread-3171 (_readerthread):
Traceback (most recent call last):
  File "c:\Users\spbsp\anaconda3\envs\ddpm\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Us

# Transform text to jsonl form


In [ ]:
# {"file_name" : "3odrUVQ9tvRpkC9II2oWzx_segment_5.png", "additional_feature": "Ellie Goulding electro"}
# So if the str before the first underscore is the same to the row track_id column, then add text to the additional feature column

{'file_name': '3odrUVQ9tvRpkC9II2oWzx_segment_5.png',
 'additional_feature': 'Ellie Goulding electro'}

SEGMENT SPECTOGRAM


Transform audio and segment spectograms


In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
from pydub import AudioSegment  # To save segments as MP3 files

def mp3_to_spectrogram(mp3_file, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        

    try:
        # Load the MP3 file using librosa
        y, sr = librosa.load(mp3_file, sr=None)  # y: signal, sr: sample rate

        # Check if the file is empty or too short
        if len(y) == 0:
            print(f"File {mp3_file} is empty or too short.")
            return

        print(f"Loaded {mp3_file}, Sample Rate: {sr}, Audio Shape: {y.shape}")
        
        # Compute the Short-Time Fourier Transform (STFT)
        stft = librosa.stft(y)
        specto = np.abs(stft)
        D = librosa.amplitude_to_db(specto, ref=np.max)
        
        # Calculate duration of the audio in seconds
        duration = librosa.get_duration(y=y, sr=sr)
        
        # Split the spectrogram into 3 equal segments
        split_duration = duration / 3  # Length of each segment in seconds

        for i in range(3):
            start_time = i * split_duration
            end_time = (i + 1) * split_duration

            # Convert start and end time to frames
            start_frame = librosa.time_to_frames(start_time, sr=sr)
            end_frame = librosa.time_to_frames(end_time, sr=sr)

            # Extract the spectrogram segment
            D_segment = D[:, start_frame:end_frame]

            # Plot and save the spectrogram segment
            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D_segment, sr=sr, x_axis='time', y_axis='log', cmap='inferno')
            plt.colorbar(format='%+2.0f dB')
            plt.title(f'Spectrogram Segment {i + 1} of {os.path.basename(mp3_file)}')
            
            # Save the spectrogram image for this segment
            output_image_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{i+1}.png")
            plt.savefig(output_image_path)
            plt.close()

            print(f"Spectrogram Segment {i+1} saved to {output_image_path}")

            # Save the corresponding audio segment as MP3
            start_ms = int(start_time * 1000)  # Convert start time to milliseconds
            end_ms = int(end_time * 1000)      # Convert end time to milliseconds
            audio_segment = AudioSegment.from_file(mp3_file)[start_ms:end_ms]

            output_audio_path = os.path.join(output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{i+1}.mp3")
            audio_segment.export(output_audio_path, format="mp3")
            print(f"Audio Segment {i+1} saved to {output_audio_path}")

    except Exception as e:
        print(f"Error processing {mp3_file}: {e}")

def process_all_mp3_in_folder(folder_path, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all MP3 files in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".mp3"):
            mp3_file = os.path.join(folder_path, file)
            print(f"Processing file: {mp3_file}")
            mp3_to_spectrogram(mp3_file, output_folder)

# Example usage for processing multiple MP3 files
folder_path = "raw/audio"  # Folder containing MP3 files
output_folder = "raw/spectograms_segmented"  # Folder to save spectrogram images and segments
process_all_mp3_in_folder(folder_path, output_folder)


Transform audio and segment spectograms
(without legends and mp3 files)

In [ ]:
import os
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

def mp3_to_spectrogram(mp3_file, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    try:
        # Load the MP3 file using librosa
        y, sr = librosa.load(mp3_file, sr=None)  # y: signal, sr: sample rate

        # Check if the file is empty or too short
        if len(y) == 0:
            print(f"File {mp3_file} is empty or too short.")
            return

        print(f"Loaded {mp3_file}, Sample Rate: {sr}, Audio Shape: {y.shape}")
        
        # Compute the Short-Time Fourier Transform (STFT)
        stft = librosa.stft(y)
        specto = np.abs(stft)
        D = librosa.amplitude_to_db(specto, ref=np.max)
        
        # Calculate duration of the audio in seconds
        duration = librosa.get_duration(y=y, sr=sr)
        
        # Split the spectrogram into 3 equal segments
        split_duration = duration / 3  # Length of each segment in seconds

        for i in range(3):
            start_time = i * split_duration
            end_time = (i + 1) * split_duration

            # Convert start and end time to frames
            start_frame = librosa.time_to_frames(start_time, sr=sr)
            end_frame = librosa.time_to_frames(end_time, sr=sr)

            # Extract the spectrogram segment
            D_segment = D[:, start_frame:end_frame]

            # Plot and save the spectrogram segment without extra elements
            plt.figure(figsize=(10, 6))
            librosa.display.specshow(D_segment, sr=sr, cmap='inferno')
            plt.axis('off')  # Disable axis
            plt.savefig(os.path.join(output_folder, f"{os.path.splitext(os.path.basename(mp3_file))[0]}_segment_{i+1}.png"),
                        bbox_inches='tight', pad_inches=0)  # Save without padding
            plt.close()

            print(f"Spectrogram Segment {i+1} saved.")

    except Exception as e:
        print(f"Error processing {mp3_file}: {e}")

def process_all_mp3_in_folder(folder_path, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all MP3 files in the folder
    for file in os.listdir(folder_path):
        if file.endswith(".mp3"):
            mp3_file = os.path.join(folder_path, file)
            print(f"Processing file: {mp3_file}")
            mp3_to_spectrogram(mp3_file, output_folder)

# Example usage for processing multiple MP3 files
folder_path = "raw/audio"  # Folder containing MP3 files
output_folder = "raw/spectrograms_csegm"  # Folder to save spectrogram images and segments
process_all_mp3_in_folder(folder_path, output_folder)


Transform spectogram to audio

In [ ]:
import numpy as np
import librosa
import librosa.display
from pydub import AudioSegment

# Function to convert spectrogram to audio and save it as MP3
def spectrogram_to_mp3(spec, sr, filename="output.mp3"):
    # Step 1: Invert the spectrogram to a time-domain audio signal
    # If the spectrogram is in dB, first convert back to amplitude
    if np.max(spec) < 0:  # Checking if it's in dB scale
        spec = librosa.db_to_amplitude(spec)
    
    # If the spectrogram is a magnitude spectrogram, use librosa's inverse STFT
    audio_signal = librosa.istft(spec)
    
    # Step 2: Save the audio signal as a WAV file
    librosa.output.write_wav("temp.wav", audio_signal, sr)
    
    # Step 3: Convert WAV to MP3 using pydub
    sound = AudioSegment.from_wav("temp.wav")
    sound.export(filename, format="mp3")
    
    print(f"Saved MP3 as {filename}")

# Example usage
if __name__ == "__main__":
    # Load an example audio file to create a spectrogram
    y, sr = librosa.load(librosa.example('trumpet'))
    
    # Generate a spectrogram from the audio signal
    spec = librosa.stft(y)
    spec_db = librosa.amplitude_to_db(np.abs(spec), ref=np.max)
    
    # Convert the spectrogram back to MP3
    spectrogram_to_mp3(spec_db, sr, filename="output.mp3")


CHECK FROM SPECTOGRAM TO MP3

In [18]:
image_path = "raw/spectrograms/0A0RBBTrgfq9eClnw6ZXT7.png"


In [ ]:
import numpy as np
import librosa
from PIL import Image
from pydub import AudioSegment
import soundfile as sf  # For saving audio files

def image_to_spectrogram(image_path):
    """
    Convert a spectrogram image to a numerical array representing the spectrogram.
    Assumes the spectrogram is grayscale, where pixel intensity represents dB values.
    """
    # Load the image
    img = Image.open(image_path).convert("L")  # Convert to grayscale ('L' mode)
    
    # Convert image to numpy array
    img_array = np.array(img)
    
    # Normalize the image to [0, 1] range (0 is minimum, 255 is maximum)
    img_array = img_array / 255.0

    # Convert the image intensities to dB scale (assuming they represent dB)
    # Rescale from [0, 1] to [-80 dB, 0 dB], as typical dB range for spectrograms is -80 to 0
    img_db = img_array * -80
    
    return img_db

def spectrogram_to_audio(spectrogram_db, sr=22050, hop_length=512):
    """
    Invert a dB-scaled spectrogram back to an audio waveform.
    """
    # Convert dB to amplitude
    spectrogram_amplitude = librosa.db_to_amplitude(spectrogram_db)
    
    # Perform the inverse Short-Time Fourier Transform (ISTFT) to recover the audio signal
    audio_signal = librosa.griffinlim(spectrogram_amplitude, hop_length=hop_length)
    
    return audio_signal

def save_as_mp3(audio_signal, sr, filename="output.mp3"):
    """
    Save the audio signal as an MP3 file using pydub.
    """
    # Save the audio signal as a temporary WAV file using soundfile
    sf.write("temp.wav", audio_signal, sr)  # Use soundfile to save as WAV
    
    # Convert the WAV file to MP3 using pydub
    sound = AudioSegment.from_wav("temp.wav")
    sound.export(filename, format="mp3")
    print(f"Saved MP3 as {filename}")

# Example usage
if __name__ == "__main__":
    # Path to the spectrogram image (PNG)
    image_path = "raw/spectrograms/0A0RBBTrgfq9eClnw6ZXT7.png"
    
    # Step 1: Convert the image to a spectrogram
    spectrogram_db = image_to_spectrogram(image_path)
    
    # Step 2: Convert the spectrogram to an audio signal
    audio_signal = spectrogram_to_audio(spectrogram_db, sr=22050)
    
    # Step 3: Save the audio signal as MP3
    save_as_mp3(audio_signal, sr=22050, filename="output.mp3")


In [1]:
import torch
torch.cuda.empty_cache()


audio shape ???
sample rate ???

Download everything in wav? or find a solution in another way ?